<a target="_blank" href="https://colab.research.google.com/github/syk-yaman/plantemoji/blob/master/Code/Colab/Plantemoji-LTSM.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
pip install influxdb-client

In [2]:
import numpy
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import logging
from scipy.signal import find_peaks
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, Dropout
from keras.callbacks import EarlyStopping
import tensorflow as tf

In [3]:
bucket = ""
org = ""
token = ""
# Store the URL of your InfluxDB instance
url=""

In [4]:
client = influxdb_client.InfluxDBClient(
   url=url,
   token=token,
   org=org
)

In [5]:
query_api = client.query_api()

In [6]:
query = 'from(bucket: "casa-mqtt-data")\
  |> range(start: 2023-02-24T00:00:00.000000000Z, stop: 2023-03-05T23:59:59.941926044Z)\
  |> filter(fn: (r) => r["_field"] == "value")\
  |> filter(fn: (r) => r["host"] == "30f2640405ed")\
  |> filter(fn: (r) => r["plant-topics"] == "student/CASA0014/plant/ucfnmyr/plantemoji/airHumidity")\
 '

In [7]:
result = query_api.query(org=org, query=query)


In [8]:
results = []
for table in result:
  for record in table.records:
    results.append((record.get_time(), record.get_value()))



In [9]:
print(len(results))
array = np.array(results)
df = pd.DataFrame(array[:, 1], columns=['humidity'])
print(df)
df.plot()

In [10]:
print(len(df)) # 401
val_percent = 0.1 # 10 percent of data
len(df)*val_percent
val_point = np.round(len(df)*val_percent)
val_index = int(len(df) - val_point)
train = df.iloc[:val_index]
val = df.iloc[val_index:]

print(len(train))
print(len(val))

plt.plot(train)
plt.plot(val)


In [11]:
scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
scaler.fit(df)
scaled_train = scaler.transform(train)
scaled_val = scaler.transform(val)

In [12]:
length = 20
batch_size = 4
n_features = 1

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

generator = TimeseriesGenerator(scaled_train, scaled_train, length=length, batch_size=batch_size)
validation_generator = TimeseriesGenerator(scaled_val, scaled_val, length=length, batch_size=batch_size)

lstm_model = Sequential()

output_space = length # Same as number of time steps in the training window

lstm_model.add(LSTM(output_space, input_shape=(length, n_features)))

#lstm_model.add(LSTM(output_space, return_sequences=True, input_shape=(length,n_features)))
#lstm_model.add(LSTM(output_space))

lstm_model.add(Dense(1))

lstm_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse')
lstm_model.fit(generator, epochs=2, validation_data=validation_generator, callbacks=[callback])

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-length:]

current_batch = first_eval_batch.reshape(1, length,n_features)


for i in range(len(val)):
  current_pred = lstm_model.predict(current_batch)[0]

  test_predictions.append(current_pred)

  current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis = 1)

true_predictions = scaler.inverse_transform(test_predictions)
val['LSTM Predictions'] = true_predictions
val.plot(figsize=(12,8))

In [ ]:
forecast = []
full_scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
scaled_full_data = full_scaler.fit_transform(df)
first_eval_batch = scaled_full_data[-length:]

current_batch = first_eval_batch.reshape(1, length, n_features)


for i in range(40):
  current_pred = lstm_model.predict(current_batch)[0]

  forecast.append(current_pred)

  current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis = 1)

In [ ]:
forecast = full_scaler.inverse_transform(forecast)

forecast_index = np.arange(401, 441, step=1)

plt.figure(figsize=(12,8))
plt.plot(df.index, df['light'])
plt.plot(forecast_index, forecast)
plt.show()